In [16]:
import os
import paramiko
import scp

from HPC_MD.Lumi_Interface import HPCConnection
from HPC_MD.Call_Runner import generate_md_runner

In [17]:
# Expand the tilde to your actual home directory path
key_path = os.path.expanduser("~/.ssh/id_ed25519")

# Now create your connection with the expanded path
hpc = HPCConnection(
    hostname="lumi.csc.fi",
    username="fuscoric",
    key_path=key_path,
    key_passphrase="Dm761pm*"
)

In [18]:
custom_config = {
    "mode": {"type": "legacy"},
    "legacy": {
        "runs": {
            "Run1": {"protein": "./protein1.pdb", "ligand": "./ligand1.sdf"},
            "Run2": {"protein": "./protein2.pdb", "ligand": "APO"}
        }
    }
}

runner = generate_md_runner(
    account="project_465001750",
    toml_config=custom_config
)
print(runner)

#!/bin/bash -l
#SBATCH --job-name=MD_Wrapper
#SBATCH --output=/dev/null
#SBATCH --error=/dev/null
#SBATCH --partition=dev-g
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=2
#SBATCH --gpus-per-node=2
#SBATCH --cpus-per-task=7
#SBATCH --time=02:00:00
#SBATCH --account=project_465001750

cat << EOF > select_gpu
#!/bin/bash

# Export the Python Path
export PATH="/scratch/project_465001750/Singularity_Envs/cheminf_rocm/env_cheminf_rocm/bin:$PATH"

# Distribute the GPU for the ranks equally
export ROCR_VISIBLE_DEVICES=$((SLURM_LOCALID % SLURM_GPUS_PER_NODE))

# Only assign GPUs to ranks > 0 (workers)
if [ $SLURM_PROCID -gt 0 ]; then
    # Adjust the calculation to account for rank 0 not using a GPU
    worker_id=$((SLURM_PROCID - 1))
    export ROCR_VISIBLE_DEVICES=$((worker_id % SLURM_GPUS_PER_NODE))
else
    # For rank 0, set an invalid GPU ID or leave it unset
    export ROCR_VISIBLE_DEVICES=""
fi

exec $*
EOF

chmod +x ./select_gpu

cat << EOF > md_config.toml
[mode]
type = "legacy"

[syste